In [1]:
%matplotlib qt

In [2]:
import matplotlib.pyplot as plt
import numpy as np
from prettyplots import pretty_axis
import pickle
from astropy.table import Table
import astropy.units as u
import os
from matplotlib.gridspec import GridSpec
import json
from scipy.io import readsav
from astropy.convolution import convolve_fft,Box1DKernel
from tqdm import tqdm

# Plot stellar SEDs

In [3]:
def bin_by_pixel(wave,flux,binsize):
    arr_len = wave.shape[0]

    _wave = wave[:arr_len-(arr_len%int(binsize))]
    _flux = flux[:arr_len-(arr_len%int(binsize))]

    binned_wave = np.mean(
        np.concatenate([[_wave[i::binsize] for i in range(binsize)]]),axis=0,
    )
    binned_flux = np.mean(
        np.concatenate([[_flux[i::binsize] for i in range(binsize)]]),axis=0,
    )

    return binned_wave,binned_flux

def bin_by_wave(wave,flux,binsize):
    new_wave = np.arange(wave[0],wave[-1],step=binsize)
    bin_inds = np.digitize(wave,new_wave)
    binned_flux = np.array([np.average(flux[bin_inds==i]) for i in tqdm(range(1,len(new_wave)))])

    return new_wave,binned_flux

alf_cen_infile = {
    'A':'./alfCenA_panchrom_SED_SISTINE_02-07-2024.fits',
    'B':'./alfCenB_panchrom_SED_SISTINE_02-07-2024.fits',
}

solar_infile = {
    'gueymard':'../atm/stellar_flux/gueymard_solar.txt',
    'woods':'../atm/stellar_flux/solar-data.idlsav',
}


alf_cen_data = {
    'A':Table.read(alf_cen_infile['A']),
    'B':Table.read(alf_cen_infile['B']),
}

solar_data = {
    'gueymard':Table.read(solar_infile['gueymard'],format='ascii',names=['wavelength','flux']),
    'woods':readsav(solar_infile['woods']),
}

# Convert units on Woods spectrum
solar_data['woods'] = {
    'wave':solar_data['woods']['wave'].flatten()*10,
    'flux':solar_data['woods']['flux'].flatten()*100
}

In [4]:
alf_cen_mask = {
    'A':(alf_cen_data['A']['flux'] > 1e-15)&(alf_cen_data['A']['wavelength'] < 4e5),
    'B':(alf_cen_data['B']['flux'] > 1e-15)&(alf_cen_data['B']['wavelength'] < 4e5),
}

mask = {
    'euv':[100,500,21],
    'fuv':[1000,3000,51],
    'visir':[3000,4e5,51]
}

for star in ['A','B']:
    for reg,wave in mask.items():
        reg_mask = np.all([alf_cen_data[star]['wavelength'] > wave[0],
                       alf_cen_data[star]['wavelength'] < wave[1],
                       alf_cen_data[star]['flux'] > 0],axis=0)
        alf_cen_data[star]['flux'][reg_mask] = convolve_fft(alf_cen_data[star]['flux'][reg_mask],Box1DKernel(wave[2]))

In [7]:
fig,ax = plt.subplots(1,1,figsize=(9,4),layout='constrained')
pretty_axis(ax)

ax.step(alf_cen_data['A']['wavelength'][alf_cen_mask['A']],
          alf_cen_data['A']['flux'][alf_cen_mask['A']],
          label=r'$\alpha$ Cen A',zorder=4)

ax.step(alf_cen_data['B']['wavelength'][alf_cen_mask['B']],
          alf_cen_data['B']['flux'][alf_cen_mask['B']],
          label=r'$\alpha$ Cen B',zorder=4)

ax.step(solar_data['woods']['wave'],
          solar_data['woods']['flux']*(1.0*u.AU/((1.33*u.pc).to(u.AU)))**2,
          label='Sun',zorder=2)

ax.set_xlim([5,4e5])
ax.set_ylim([1e-16,5e-8])
ax.set_xscale('log')
ax.set_yscale('log')
ax.legend(fontsize=14)
ax.set_xlabel(r'Wavelength [$\rm{\AA}$]',fontsize=18)
ax.set_ylabel(r'Flux [erg s$^{-1}$ cm$^{-2}$ nm$^{-1}$]',fontsize=18)

Text(0, 0.5, 'Flux [erg s$^{-1}$ cm$^{-2}$ nm$^{-1}$]')

In [86]:
new_wave = np.arange(alf_cen_data['A']['wavelength'][0],
                     alf_cen_data['A']['wavelength'][-1],
                     step=5)
bin_inds = np.digitize(alf_cen_data['A']['wavelength'],new_wave)
plt.figure()
plt.plot(alf_cen_data['A']['wavelength'],bin_inds)

In [20]:
plt.savefig('../paper-figures/stellar_seds.png',format='png')
plt.savefig('../paper-figures/stellar_seds.pdf',format='pdf')

# Plot atmosphere retrievals

In [9]:
with open('./label_locs.json') as file:
    label_locs = json.load(file)

## Terrestrial

In [5]:
min_file = '../output/terrestrial/alf_cen_min.vul'
peak_file = '../output/terrestrial/alf_cen_peak.vul'
earth_file = '../output/Earth.vul'

In [22]:
specs_to_plot = ['H','H2','H2O','CO','CO2','CH4','N2','O','O2']

fig = plt.figure(layout='constrained',figsize=(12,6))

gs = GridSpec(1,3,figure=fig)
ax = [
    fig.add_subplot(gs[0:2]),
    fig.add_subplot(gs[2])
]

for subax in ax:
    pretty_axis(subax)

for infile,linestyle,label in zip([peak_file,min_file,earth_file],['-','--',':'],['peak','min','Earth']):
    #if not label=='peak':
    #    continue
    with open(infile,'rb') as handle:
        data = pickle.load(handle)
        pressure = data['atm']['pco']/1.e6
        
    mixing_ratios = {}
    
    for species in specs_to_plot:
        ind = data['variable']['species'].index(species)
        mixing_ratios[species] = data['variable']['ymix'][:,ind]

    i = 0
    for sp,mr in mixing_ratios.items():
        for char in sp:
            if char.isdigit():
                # Format species for label
                sp = sp.replace(char,"$_%s$"%(char))
        if 'peak' in infile:
            ax[0].plot(mr,pressure,label=sp,ls=linestyle,c='C%s'%(i))
        else:
            ax[0].plot(mr,pressure,ls=linestyle,c='C%s'%(i))
        i += 1

ax[0].set_yscale('log')
ax[0].set_xscale('log')
ax[0].invert_yaxis() 
ax[0].minorticks_on()
ax[0].set_ylim((data['atm']['pco'][0]/1e6,data['atm']['pco'][-1]/1e6))
ax[0].set_ylabel("Pressure [bar]",fontsize=20)
ax[0].set_xlabel("Mixing ratio",fontsize=20)
ax[0].legend(fontsize=14)

'''
# Plot SEDs
ax[1].loglog(solar_data['woods']['wavelength'],solar_data['woods']['flux'],label='Woods solar')
ax[1].loglog(alf_cen_data['peak']['wavelength'],alf_cen_data['peak']['flux'],label=r'$\alpha$ Cen peak')
ax[1].loglog(alf_cen_data['min']['wavelength'],alf_cen_data['min']['flux'],label=r'$\alpha$ Cen min')
ax[1].legend(fontsize=12)
ax[1].set_xlabel(r'Wavelength [nm]',fontsize=18)
ax[1].set_ylabel(r'Stellar lux [erg s$^{-1}$ cm$^{-2}$ nm$^{-1}$]',fontsize=18)
'''

# Plot TP profile
ax[1].semilogy(data['atm']['Tco'], pressure)
ax[1].set_xlabel('Temperature [K]',fontsize=18)
ax[1].invert_yaxis()
ax[1].sharey(ax[0])
ax[1].tick_params(axis='y',labelleft=False)

ax[0].plot([1e-12,1e-10],[1e-1,1e-1],ls='-',c='k')
ax[0].text(s='Peak',x=1.5e-10,y=1e-1,fontsize=14,verticalalignment='center')

ax[0].plot([1e-12,1e-10],[2e-1,2e-1],ls='--',c='k')
ax[0].text(s='Min',x=1.5e-10,y=2e-1,fontsize=14,verticalalignment='center')

ax[0].plot([1e-12,1e-10],[4e-1,4e-1],ls=':',c='k')
ax[0].text(s='Earth',x=1.5e-10,y=4e-1,fontsize=14,verticalalignment='center')

Text(1.5e-10, 0.4, 'Earth')

In [23]:
plt.savefig('../paper-figures/atm_terrestrial.png',format='png')
plt.savefig('../paper-figures/atm_terrestrial.pdf',format='pdf')

In [17]:
fig,ax = plt.subplots(1,1,layout='constrained',figsize=(8,6))
pretty_axis(ax)

for species in ['H2O','CO2','CH4']:                
    with open(peak_file,'rb') as handle:
        data_peak = pickle.load(handle)
    with open(min_file,'rb') as handle:
        data_min = pickle.load(handle)

    ind = {
        'peak':data_peak['variable']['species'].index(species),
        'min':data_min['variable']['species'].index(species),
    }

    pressure = data_peak['atm']['pco']/1e6
    mr_ratio = data_min['variable']['ymix'][:,ind['min']]/data_peak['variable']['ymix'][:,ind['peak']]
    maxdiff_ind = np.argmax(mr_ratio)

    peak_mr = data_peak['variable']['ymix'][maxdiff_ind,ind['peak']]
    min_mr = data_min['variable']['ymix'][maxdiff_ind,ind['min']]

    peak_ratio = min_mr/peak_mr
    peak_pressure = pressure[maxdiff_ind]

    for char in species:
            if char.isdigit():
                # Format species for label
                species = species.replace(char,"$_%s$"%(char))

    ax.semilogx(
        pressure,
        mr_ratio,
        label=species)
    
    print(f'-----{species}-----')
    print(ind)
    print(f'Peak MR: {peak_mr:0.4e}')
    print(f'Min MR: {min_mr:0.4e}')
    print(f'MR ratio: {peak_ratio:0.4e}')
    print(f'Pressure: {peak_pressure:0.4e}')

ax.set_xlabel('Pressure [bar]',fontsize=18)
ax.set_ylabel(r'MR$_\mathrm{min}$ / MR$_\mathrm{peak}$',fontsize=19)
ax.invert_xaxis()
ax.legend(fontsize=14)
ax.set_xlim([1e-4,5e-8])

-----H$_2$O-----
{'peak': 2, 'min': 2}
Peak MR: 5.1138e-09
Min MR: 2.7299e-08
MR ratio: 5.3384e+00
Pressure: 5.0000e-08
-----CO$_2$-----
{'peak': 18, 'min': 18}
Peak MR: 3.3846e-06
Min MR: 1.3285e-05
MR ratio: 3.9252e+00
Pressure: 5.0000e-08
-----CH$_4$-----
{'peak': 9, 'min': 9}
Peak MR: 2.1707e-10
Min MR: 1.5306e-09
MR ratio: 7.0511e+00
Pressure: 5.0000e-08


(0.0001, 5e-08)

## Gas giant

In [7]:
infiles = {
    'min':'../output/gas-giant/alf_cen_min.vul',
    'peak':'../output/gas-giant/alf_cen_peak.vul'
}

In [28]:
specs_to_plot = ['H','He','H2','H2O','CO','CO2','CH4','N2','O','O2']

fig = plt.figure(layout='constrained',figsize=(12,6))

gs = GridSpec(1,3,figure=fig)

ax = [
    fig.add_subplot(gs[0:2]),
    fig.add_subplot(gs[2])
]

for subax in ax:
    pretty_axis(subax)

for profile,ls in zip(['min','peak'],['--','-']):
    with open(infiles[profile],'rb') as handle:
        data = pickle.load(handle)
        pressure = data['atm']['pco']/1.e6
        
    mixing_ratios = {}
    
    for species in specs_to_plot:
        ind = data['variable']['species'].index(species)
        mixing_ratios[species] = data['variable']['ymix'][:,ind]

    i = 0
    for sp,mr in mixing_ratios.items():
        for char in sp:
            if char.isdigit():
                # Format species for label
                sp = sp.replace(char,"$_%s$"%(char))
        if profile=='peak':
            ax[0].plot(mr,pressure,label=sp,ls=ls,c='C%s'%(i))
        else:
            ax[0].plot(mr,pressure,ls=ls,c='C%s'%(i))
        i += 1

for i,(species,loc) in enumerate(label_locs['gas'].items()):
    if species in ['H','O$_2$']:
        ax[0].text(s=species,x=loc[0],y=loc[1],
                   color=f"C{i}",
                   fontsize=14,
                   verticalalignment='center')

ax[0].set_yscale('log')
ax[0].set_xscale('log')
ax[0].invert_yaxis() 
ax[0].minorticks_on()
ax[0].set_ylim((data['atm']['pco'][0]/1e6,data['atm']['pco'][-1]/1e6))
ax[0].set_ylabel("Pressure [bar]",fontsize=20)
ax[0].set_xlabel("Mixing ratio",fontsize=20)
ax[0].legend(fontsize=14)

# Plot TP profile
ax[1].semilogy(data['atm']['Tco'], pressure)
ax[1].set_xlabel('Temperature [K]',fontsize=18)
ax[1].invert_yaxis()
ax[1].sharey(ax[0])
ax[1].tick_params(labelleft=False)

ax[0].minorticks_on()
ax[1].minorticks_on()

ax[0].plot([1e-17,1e-15],[1e-1,1e-1],ls='-',c='k')
ax[0].text(s='Peak',x=3e-15,y=1e-1,fontsize=14,verticalalignment='center')

ax[0].plot([1e-17,1e-15],[2e-1,2e-1],ls='--',c='k')
ax[0].text(s='Min',x=3e-15,y=2e-1,fontsize=14,verticalalignment='center')

Text(3e-15, 0.2, 'Min')

In [29]:
plt.savefig('../paper-figures/atm_gas-giant.png',format='png')
plt.savefig('../paper-figures/atm_gas-giant.pdf',format='pdf')